In [3]:
# Step 1: Imports and setup
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms, models
import numpy as np
from tqdm import tqdm
from PIL import Image

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cpu


In [4]:
# 🔥 Step 0 — Clean .ipynb_checkpoints automatically from all dataset folders

import os
import shutil

def clean_checkpoints(base_folder):
    removed = 0
    for root, dirs, files in os.walk(base_folder):
        for d in dirs:
            if d == '.ipynb_checkpoints':
                path = os.path.join(root, d)
                shutil.rmtree(path, ignore_errors=True)
                print("🧹 Removed:", path)
                removed += 1
    if removed == 0:
        print("✅ No .ipynb_checkpoints found.")
    else:
        print(f"✅ Cleaned {removed} folders successfully!")

clean_checkpoints("dataset")


✅ No .ipynb_checkpoints found.


In [5]:
# Step 2: Load dataset and take small subset for faster training

IMG_SIZE = 224
BATCH = 8  # small batch for CPU

# Image transforms (resize + normalize)
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Load full datasets
train_data_full = datasets.ImageFolder("dataset/train", transform=train_transform)
val_data = datasets.ImageFolder("dataset/val", transform=test_transform)
test_data = datasets.ImageFolder("dataset/test", transform=test_transform)

# ✅ Take only a small random subset for training
subset_size = 3000
indices = np.random.choice(len(train_data_full), subset_size, replace=False)
train_data = Subset(train_data_full, indices)

# Dataloaders
train_loader = DataLoader(train_data, batch_size=BATCH, shuffle=True)
val_loader = DataLoader(val_data, batch_size=BATCH, shuffle=False)
test_loader = DataLoader(test_data, batch_size=BATCH, shuffle=False)

print("Classes:", train_data_full.classes)
print(f"✅ Training with {subset_size} images (subset)")


Classes: ['Fake', 'Real']
✅ Training with 3000 images (subset)


In [6]:
# Step 3: Load Pretrained ResNet18 for Transfer Learning

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

# Load pre-trained ResNet18
model = models.resnet18(pretrained=True)

# Freeze all layers except the final fully-connected layer
for param in model.parameters():
    param.requires_grad = False

# Replace final layer for 2-class classification (Fake vs Real)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)  # 2 outputs
model = model.to(device)

# Define loss function & optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

print("✅ ResNet18 model ready for training")


C:\Users\LENOVO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\LENOVO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


✅ ResNet18 model ready for training


In [8]:
from tqdm import tqdm
import torch
import numpy as np

# Limit validation subset for faster run
val_subset_size = min(1000, len(val_data))
val_indices = np.random.choice(len(val_data), val_subset_size, replace=False)
val_subset = torch.utils.data.Subset(val_data, val_indices)
val_loader = DataLoader(val_subset, batch_size=BATCH, shuffle=False)

EPOCHS = 3
best_val_acc = 0

for epoch in range(EPOCHS):
    
    # ---------------------- TRAINING --------------------------
    print(f"\n🧠 Training Epoch {epoch+1}/{EPOCHS}...")
    model.train()
    running_loss = 0.0

    for images, labels in tqdm(train_loader, desc=f"Train {epoch+1}"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # ---------------------- VALIDATION ------------------------
    print(f"🔍 Validating Epoch {epoch+1}...")
    model.eval()
    correct = 0
    total = 0
    val_loss = 0.0

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="Validating", leave=False):
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_acc = 100 * correct / total

    # ---------------------- PRINT EPOCH RESULT ------------------------
    print(f"\n✅ Epoch [{epoch+1}/{EPOCHS}] | "
          f"Train Loss: {running_loss/len(train_loader):.4f} | "
          f"Val Loss: {val_loss/len(val_loader):.4f} | "
          f"Val Acc: {val_acc:.2f}%")

    # ---------------------- SAVE BEST MODEL ---------------------------
    if val_acc > best_val_acc:
        torch.save(model.state_dict(), "mini_deepfake_model.pth")
        best_val_acc = val_acc
        print("💾 Model saved (best so far)")



🧠 Training Epoch 1/3...


Train 1: 100%|███████████████████████████████████████████████████████████████████████| 375/375 [02:31<00:00,  2.47it/s]


🔍 Validating Epoch 1...



✅ Epoch [1/3] | Train Loss: 0.5278 | Val Loss: 0.5690 | Val Acc: 70.50%
💾 Model saved (best so far)

🧠 Training Epoch 2/3...


Train 2: 100%|███████████████████████████████████████████████████████████████████████| 375/375 [02:31<00:00,  2.47it/s]


🔍 Validating Epoch 2...



✅ Epoch [2/3] | Train Loss: 0.5239 | Val Loss: 0.4260 | Val Acc: 80.70%
💾 Model saved (best so far)

🧠 Training Epoch 3/3...


Train 3: 100%|███████████████████████████████████████████████████████████████████████| 375/375 [02:32<00:00,  2.46it/s]


🔍 Validating Epoch 3...



✅ Epoch [3/3] | Train Loss: 0.5402 | Val Loss: 0.4853 | Val Acc: 76.10%


In [14]:
# FAST TEST EVALUATION (only 1000 images)
import numpy as np
from torch.utils.data import Subset

# Take 1000 random test images
test_subset_size = min(1000, len(test_data))
test_indices = np.random.choice(len(test_data), test_subset_size, replace=False)
test_subset = Subset(test_data, test_indices)
test_loader_fast = DataLoader(test_subset, batch_size=BATCH, shuffle=False)

model.load_state_dict(torch.load("mini_deepfake_model.pth", map_location=device))
model.eval()

correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader_fast:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"🎯 FAST TEST Accuracy on 1000 images: {100 * correct / total:.2f}%")


🎯 FAST TEST Accuracy on 1000 images: 67.40%


In [15]:
print(train_data_full.classes)


['Fake', 'Real']


In [16]:
 predict_image("path/to/test/img.jpg")


NameError: name 'predict_image' is not defined